In [ ]:
import os
import numpy as np
import librosa
import soundfile as sf

# Paths
input_folder = '/home/masense/projects/ai8x-training/data/AudioAutoencoder/raw/fan/id_00/normal'
output_folder = '/home/masense/projects/ai8x-training/data/AudioAutoencoder/raw/fan/id_chunked00'
os.makedirs(output_folder, exist_ok=True)

def process_audio_file(file_path):
    """
    Process a single audio file: load, compute Mel spectrogram, pad/crop, inverse transform, and save.
    """
    # Load the audio file
    audio, sr = librosa.load(file_path, sr=16000)
    mel_spectrogram = librosa.feature.melspectrogram(
        y=audio,
        sr=sr,
        n_mels=64,
        n_fft=1024,
        hop_length=512,
        power=2.0
    )
    
    # Convert to dB
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
    
    # Target size [128, 128]
    def pad_or_crop_spectrogram(mel_spectrogram_db, target_height=128, target_width=128):
        h, w = mel_spectrogram_db.shape
        if h < target_height:
            mel_spectrogram_db = np.pad(mel_spectrogram_db, ((0, target_height - h), (0, 0)), mode='constant')
        elif h > target_height:
            mel_spectrogram_db = mel_spectrogram_db[:target_height, :]
            
        if w < target_width:
            mel_spectrogram_db = np.pad(mel_spectrogram_db, ((0, 0), (0, target_width - w)), mode='constant')
        elif w > target_width:
            mel_spectrogram_db = mel_spectrogram_db[:, :target_width]
            
        return mel_spectrogram_db

    processed_spectrogram = pad_or_crop_spectrogram(mel_spectrogram_db)
    mel_spectrogram_power = librosa.db_to_power(processed_spectrogram)
    audio_reconstructed = librosa.feature.inverse.mel_to_audio(mel_spectrogram_power, sr=sr)

    # Save 
    file_name = os.path.basename(file_path)
    output_file = os.path.join(output_folder, file_name)
    
    sf.write(output_file, audio_reconstructed, sr)
    print(f"Saved processed audio to {output_file}")

def process_all_files(input_folder, output_folder):
    """
    Process all audio files in the input folder and save results to the output folder.
    """
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.wav'):
            file_path = os.path.join(input_folder, file_name)
            process_audio_file(file_path)

# Run the processing
process_all_files(input_folder, output_folder)